In [46]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score

In [79]:
#Createing the connection to the database part 1
mongo = MongoClient(port=27017)
db = mongo['hockey_project_db']
db.list_collection_names()

['2022-2023', '2023-2024']

In [80]:
#creating connections part 2: putting it into a dataframe
data_2022_2023 = db['2022-2023']
answer = list(data_2022_2023.find({}))
df_2022 = pd.json_normalize(answer)

In [81]:
df_2022.head()

,_id,,playerId,season,name,team,position,situation,games_played,minutes/game,...,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts,Line_or_Pair
0,65a0837f1824a1bb36f2b6dd,0,8471817,2022,Ryan Reaves,MIN,R,all,73,9.32,...,29,7,1.16,3.58,1.96,0,4,1,69,3rd line
1,65a0837f1824a1bb36f2b6de,1,8480950,2022,Ilya Lyubushkin,BUF,D,all,68,15.01,...,2,0,1.09,0.28,0.00,1,1,0,52,3rd pair
2,65a0837f1824a1bb36f2b6df,2,8475625,2022,Matt Irwin,WSH,D,all,61,13.04,...,5,0,1.98,0.46,0.00,2,0,0,102,3rd pair
3,65a0837f1824a1bb36f2b6e0,3,8480860,2022,Kevin Bahl,NJD,D,all,42,14.02,...,1,0,1.13,0.08,0.00,1,1,0,47,3rd pair
4,65a0837f1824a1bb36f2b6e1,4,8477952,2022,Robby Fabbri,DET,C,all,28,16.00,...,14,10,1.07,1.73,4.02,1,2,4,53,2nd line


In [83]:
df_2022.drop(['_id', '', 'playerId','season','name','team','situation'], axis='columns',inplace=True)
df_2022.head()

,position,games_played,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotAttempts,I_F_points,...,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_unblockedShotAttempts,Line_or_Pair
0,R,73,9.32,49.47,6.70,3.80,5,5,79,15,...,29,7,1.16,3.58,1.96,0,4,1,69,3rd line
1,D,68,15.01,37.10,1.36,1.83,6,6,78,14,...,2,0,1.09,0.28,0.00,1,1,0,52,3rd pair
2,D,61,13.04,69.79,2.44,3.85,1,2,140,5,...,5,0,1.98,0.46,0.00,2,0,0,102,3rd pair
3,D,42,14.02,32.17,1.21,1.81,2,4,79,8,...,1,0,1.13,0.08,0.00,1,1,0,47,3rd pair
4,C,28,16.00,39.69,6.81,2.85,5,4,65,16,...,14,10,1.07,1.73,4.02,1,2,4,53,2nd line


In [84]:
df_2022.to_csv('model_2022-2023.csv')

# Preparing the Data

In [85]:
# Apply get_dummies on position and line/pair columns
dummies = pd.get_dummies(df_2022[['position','Line_or_Pair']])
# Drop the columns that get_dummies was used on
df_2022_col_drop = df_2022.drop(['position','Line_or_Pair'], axis=1)

# Concatenate the dummies df with the original df that had the columns dropped
df_2022_prepped = pd.concat([df_2022_col_drop, dummies], axis=1)

df_2022_prepped.head()

,games_played,minutes/game,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotAttempts,I_F_points,I_F_goals,...,position_C,position_D,position_L,position_R,Line_or_Pair_1st line,Line_or_Pair_1st pair,Line_or_Pair_2nd line,Line_or_Pair_2nd pair,Line_or_Pair_3rd line,Line_or_Pair_3rd pair
0,73,9.32,49.47,6.70,3.80,5,5,79,15,5,...,0,0,0,1,0,0,0,0,1,0
1,68,15.01,37.10,1.36,1.83,6,6,78,14,2,...,0,1,0,0,0,0,0,0,0,1
2,61,13.04,69.79,2.44,3.85,1,2,140,5,2,...,0,1,0,0,0,0,0,0,0,1
3,42,14.02,32.17,1.21,1.81,2,4,79,8,2,...,0,1,0,0,0,0,0,0,0,1
4,28,16.00,39.69,6.81,2.85,5,4,65,16,7,...,1,0,0,0,0,0,1,0,0,0


In [86]:
#Splitting the data
X = df_2022_prepped.copy()
X.drop('I_F_points', axis=1, inplace=True)
y = df_2022_prepped['I_F_points'].ravel()

In [87]:
#further splitting into test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12)

In [88]:
X_train.dtypes

games_played                   int64
minutes/game                 float64
I_F_xOnGoal                  float64
I_F_xGoals                   float64
I_F_xRebounds                float64
I_F_primaryAssists             int64
I_F_secondaryAssists           int64
I_F_shotAttempts               int64
I_F_goals                      int64
I_F_rebounds                   int64
I_F_reboundGoals               int64
penalties                      int64
I_F_penalityMinutes            int64
I_F_hits                       int64
I_F_takeaways                  int64
I_F_giveaways                  int64
I_F_lowDangerShots             int64
I_F_mediumDangerShots          int64
I_F_highDangerShots            int64
I_F_lowDangerxGoals          float64
I_F_mediumDangerxGoals       float64
I_F_highDangerxGoals         float64
I_F_lowDangerGoals             int64
I_F_mediumDangerGoals          int64
I_F_highDangerGoals            int64
I_F_unblockedShotAttempts      int64
position_C                     uint8
p

In [89]:
#creating the scaler variable
scaler = StandardScaler()

In [90]:
X_scaler = scaler.fit(X_train)

In [91]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Linear Regression Model

In [92]:
linear_model = LinearRegression()

In [93]:
# Fit the data into the model
linear_model.fit(X_train_scaled, y_train)

LinearRegression()

In [94]:
# Display the model's best fit line formula
print(f"Model's formula: y = {linear_model.intercept_} + {linear_model.coef_[0]}X")

Model's formula: y = 28.525862068965516 + 7.746433318244927e-15X


In [96]:
#Make predictions with the model using X_test_scaled
predicted_y_values = linear_model.predict(X_test_scaled)
predicted_y_values

array([1.50000000e+01, 3.00000000e+01, 9.00000000e+00, 2.70000000e+01,
       2.10000000e+01, 2.10000000e+01, 3.50000000e+01, 4.30000000e+01,
       2.10000000e+01, 2.50000000e+01, 5.50000000e+01, 1.80000000e+01,
       5.40000000e+01, 3.30000000e+01, 1.10000000e+01, 1.20000000e+01,
       8.00000000e+00, 4.10000000e+01, 6.00000000e+00, 3.00000000e+00,
       1.05000000e+02, 1.70000000e+01, 4.00000000e+00, 4.00000000e+00,
       1.50000000e+01, 8.00000000e+00, 8.00000000e+00, 1.30000000e+01,
       2.00000000e+01, 5.00000000e+00, 4.60000000e+01, 3.00000000e+01,
       1.20000000e+01, 1.00000000e+01, 3.60000000e+01, 1.00000000e+00,
       1.60000000e+01, 5.00000000e+00, 1.00000000e+00, 1.50000000e+01,
       1.70000000e+01, 5.00000000e+01, 2.10000000e+01, 2.00000000e+00,
       1.09000000e+02, 6.00000000e+00, 2.60000000e+01, 3.50000000e+01,
       8.30000000e+01, 5.00000000e+00, 5.50000000e+01, 1.20000000e+01,
       3.80000000e+01, 2.20000000e+01, 1.00000000e+01, 1.40000000e+01,
      

In [97]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = linear_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predicted_y_values)
mse = mean_squared_error(y_test, predicted_y_values)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

The score is -105.3723059892478.
The r2 is 1.0.
The mean squared error is 8.779289942311203e-28.
The root mean squared error is 2.9629866591517424e-14.
The standard deviation is 24.933234793302383.


c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


# Random Forest

In [98]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=12)

In [99]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [100]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [101]:
# Compute metrics for the linear regression model: score, r2, mse, rmse, std
score = rf_model.score(X_test, y_test, sample_weight=None)
r2 = r2_score(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
std = np.std(y_test)

# Print relevant metrics.
print(f"The score is {score}.")
print(f"The r2 is {r2}.")
print(f"The mean squared error is {mse}.")
print(f"The root mean squared error is {rmse}.")
print(f"The standard deviation is {std}.")

c:\Users\evan-\anaconda3\envs\PythonData\lib\site-packages\sklearn\base.py:444: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


The score is 0.005154639175257732.
The r2 is 0.8919846618132584.
The mean squared error is 67.14948453608247.
The root mean squared error is 8.194478905707333.
The standard deviation is 24.933234793302383.
